In [2]:
import pandas as pd 
import numpy as np 
import preprocessing
import tfidf
import scipy
import joblib

pd.set_option('max_colwidth', 5000)

In [3]:
#leggi db stack overflow 
#answers = pd.read_csv("DB/Answers.csv", sep=",",low_memory = False ,encoding='latin-1')
tags = pd.read_csv("DB/Tags.csv", sep=";", low_memory = False)
questions = pd.read_csv("DB/Questions.csv", sep=",", low_memory = False, encoding='latin-1')

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn
...,...,...
1048570,12874050,dbpedia
1048571,12874050,spotlight-dbpedia
1048572,12874100,vb.net
1048573,12874100,mouse


In [ ]:
# droppa le colonne data e owner user id che non servono e prendi solo le prime 100 righe
#answers = answers.head(100)
#answers.drop("CreationDate", axis=1, inplace=True)
#answers.drop("OwnerUserId", axis=1, inplace=True)
#Ordino in base al parent id relativo alle domande 
#answers.sort_values(by=['ParentId','Score'], ascending = False, inplace = True)


In [ ]:
#raggruppa i tag per id di domanda, in modo da avere una lista di tag per ogni domanda 
tags = tags.groupby('Id').agg(lambda x: x.tolist())
tags = tags.head(100)
tags
tags.Tag.apply(str)
type(tags)

In [ ]:
#Trasformo i tag in una stringa divisa da ','
tags['Tag'] = [','.join(map(str, l)) for l in tags['Tag']]
tags

In [ ]:
# droppa le colonne relative alle date che non servono e prendi solo le prime 100 righe
questions.drop("CreationDate", axis=1, inplace=True)
questions.drop("ClosedDate", axis=1, inplace = True)
questions.drop("OwnerUserId", axis=1, inplace = True)





In [ ]:
questions = questions.head(10000)
questions

In [ ]:
#Merge tra le colonne titolo e body in un unica colonna 
#questions["Body"] = questions["Title"] + questions["Body"] 
#questions.drop("Title", axis=1, inplace = True)
questions

In [ ]:
#Pulizia della colonna body 
questions_processed = questions["Title"] + questions["Body"]
questions.drop("Title", axis=1, inplace = True)
questions_processed = preprocessing.remove_tags(questions_processed)
questions["Body"] = questions_processed
questions



In [ ]:
docs = questions['Body'].apply(lambda x: preprocessing.clear_text(x))
docs
#docs.to_csv('DB/questions_cleared.csv')
questions

In [ ]:
question_tagged = pd.merge(questions, tags, on='Id')
question_tagged
question_tagged.to_csv('DB/questions_tagged.csv')


In [ ]:

question_sample = questions_processed.sample(100, random_state= 10).sort_index(axis=0)

question_sample


In [9]:
#Crea matrice word count per calcolo tfiidf
#Prova con secondo db 

data = pd.read_csv('DB/Preprocessed_data.csv')

data = data[data['processed_title'].notna()]

docs = data["post_corpus"]
tfidf.get_word_count_vec(docs)


In [21]:
data = data[data['processed_title'].notna()]

question = data.iloc[[65719]].title.values[0]
tags = data.iloc[[65719]].tags
tags
#titles = data["processed_title"]
#titles

65723    list|python-3.x|dictionary|zip
Name: tags, dtype: object

In [11]:
data.iloc[[2633]].title

2633    Combine model data with list of objects
Name: title, dtype: object

In [20]:
#Prove di tf idf 
matr = scipy.sparse.load_npz('DB/wc_matrix.npz')
cvec = joblib.load("DB/count_vec.pkl")
#for txt in question_sample:
    #out = tfidf.tf_idf(matr,cvec,txt)
    #print(out)
out = tfidf.tf_idf(matr,cvec,question)
print(out)

[['list', 0.839313907602809], ['dictionary', 0.3063028646088001], ['multiple', 0.2928006892811161], ['string', 0.25053648234088405], ['create', 0.230716161136808], ['propertycoefficient', 0.0], ['propertirs', 0.0], ['propertiy', 0.0]]


C:\Users\andri\.virtualenvs\Progetto_StackOverflow-UQruahqZ\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
df =  pd.read_csv("DB/questions_cleared.csv", sep=",", low_memory = False, encoding='latin-1')


#for i, row in df.iterrows('Body'):
#    print(type(row))